# 🧠 ContextDecay: Advanced Memory Management & Cost Optimization

Welcome to the complete guide to ContextDecay! This notebook teaches you how to build smart memory management systems that automatically optimize agent performance and costs.

## 🎯 Learning Objectives

Master the art of agent memory management:
- **Temporal decay models**: How memories naturally fade over time
- **Importance-based retention**: Keep critical information longer
- **Cost optimization**: Balance memory richness with budget constraints
- **Performance tuning**: Scale memory systems for production workloads
- **Real-world applications**: Conversational AI, multi-session agents, and more

---

## 🚀 Setup & Installation

In [ ]:
# Install Argentum if needed
import sys
import subprocess

try:
    import argentum
    print("✅ Argentum already installed")
except ImportError:
    print("📦 Installing Argentum...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "argentum-agent"])
    print("✅ Installation complete!")

# Import required modules
from argentum import ContextDecay
from datetime import datetime, timedelta
import json
import random
import time
import math
import matplotlib.pyplot as plt
import numpy as np

print(f"🧠 ContextDecay Tutorial - Argentum v{argentum.__version__}")
print(f"📅 Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 📚 Chapter 1: Understanding Temporal Decay

Learn the mathematical foundations and practical implications of memory decay in AI agents.

In [ ]:
print("📚 Chapter 1: Understanding Temporal Decay")
print("="*45)

# Mathematical foundation
print("🧮 Mathematical Foundation:")
print("The default decay function follows exponential decay:")
print("   current_weight = importance × (0.5 ^ (steps_elapsed / half_life))")
print("")
print("This ensures that after 'half_life' steps, an item retains 50% of its original importance.")

# Demonstrate basic decay with visualization
print("\n📊 Decay Visualization:")

def exponential_decay(importance, steps, half_life):
    """Standard exponential decay function"""
    return importance * (0.5 ** (steps / half_life))

# Different half-life scenarios
half_lives = [5, 10, 20]
importance = 1.0
max_steps = 30

print(f"Tracking decay over {max_steps} steps for different half-lives:")
print("Steps | Half-life=5 | Half-life=10 | Half-life=20")
print("-" * 50)

for step in range(0, max_steps + 1, 5):
    weights = [exponential_decay(importance, step, hl) for hl in half_lives]
    print(f"{step:5d} | {weights[0]:11.3f} | {weights[1]:12.3f} | {weights[2]:12.3f}")

# Practical example: Conversational AI memory
print("\n💬 Practical Example: Conversational AI Memory")
print("-" * 45)

# Initialize conversation agent with memory
conversation_memory = ContextDecay(half_life_steps=8)

print("🤖 Simulating a conversation about vacation planning:")

# Conversation simulation
conversation_turns = [
    ("user_name", "Sarah", 0.9, "Hi, I'm Sarah and I need help planning a vacation"),
    ("destination_interest", "Europe", 0.8, "I'm thinking about Europe, maybe Italy or France"),
    ("budget_range", "3000_5000", 0.9, "My budget is between $3000-5000"),
    ("travel_dates", "June_2024", 0.9, "I want to travel in June 2024"),
    ("group_size", "couple", 0.7, "It's just me and my partner"),
    ("accommodation_pref", "boutique_hotels", 0.6, "We prefer boutique hotels over chains"),
    ("activity_interest", "culture_food", 0.8, "We're interested in culture and food experiences"),
    ("specific_destination", "Tuscany", 1.0, "Actually, let's focus on Tuscany specifically"),
    ("duration", "10_days", 0.8, "We're thinking about 10 days"),
    ("transport_pref", "rental_car", 0.7, "We'd like to rent a car to explore")
]

for turn, (key, value, importance, description) in enumerate(conversation_turns):
    conversation_memory.add(key, value, importance=importance)
    print(f"Turn {turn + 1:2d}: {description} (importance: {importance})")
    
    # Advance time to simulate conversation progression
    conversation_memory.step()

print(f"\n🧠 Memory State After {len(conversation_turns)} Turns:")
stats = conversation_memory.get_stats()
print(f"   Total memories: {stats['total_items']}")
print(f"   Active memories: {stats['active_items']} (threshold: 0.3)")
print(f"   Average decay: {stats['avg_decay']:.3f}")
print(f"   Oldest memory age: {stats['oldest_age']} turns")

# Show what's still remembered
active_memories = conversation_memory.get_active(threshold=0.3)
print("\n💭 Still Active in Memory:")
for key, value, weight in active_memories:
    original_importance = next(imp for k, v, imp, desc in conversation_turns if k == key)
    print(f"   {key:20s}: '{value}' (weight: {weight:.3f}, was: {original_importance})")

# Demonstrate the effect of importance on retention
print("\n🎯 Key Insight: High-importance items persist longer")
high_importance_items = [item for item in active_memories if item[2] > 0.5]
print(f"   Items with weight > 0.5: {len(high_importance_items)}/{len(active_memories)}")

critical_info = ["specific_destination", "budget_range", "travel_dates"]
retained_critical = [key for key, _, _ in active_memories if key in critical_info]
print(f"   Critical info retained: {len(retained_critical)}/{len(critical_info)}")
print(f"   Retained: {', '.join(retained_critical)}")

## ⚙️ Chapter 2: Custom Decay Functions

Learn to implement custom decay functions for specialized use cases.

In [ ]:
print("⚙️ Chapter 2: Custom Decay Functions")
print("="*40)

print("🎨 Custom Decay Functions for Different Use Cases:")

# Define various decay functions
def linear_decay(importance, steps, half_life):
    """Linear decay - steady decline"""
    decline_rate = importance / (half_life * 2)  # Reach ~0 at 2x half_life
    return max(0, importance - (steps * decline_rate))

def logarithmic_decay(importance, steps, half_life):
    """Logarithmic decay - fast initial decline, then slow"""
    if steps == 0:
        return importance
    decay_factor = math.log(steps + 1) / math.log(half_life + 1)
    return importance * max(0, 1 - decay_factor)

def sigmoid_decay(importance, steps, half_life):
    """Sigmoid decay - sudden drop around half_life"""
    steepness = 0.5
    sigmoid_value = 1 / (1 + math.exp(steepness * (steps - half_life)))
    return importance * sigmoid_value

def stepped_decay(importance, steps, half_life):
    """Stepped decay - discrete reductions at intervals"""
    step_interval = half_life / 2
    reduction_per_step = 0.25
    num_steps = int(steps / step_interval)
    return importance * max(0, 1 - (num_steps * reduction_per_step))

# Demonstrate different decay functions
print("\n📊 Comparing Decay Functions:")
decay_functions = {
    "Exponential": exponential_decay,
    "Linear": linear_decay,
    "Logarithmic": logarithmic_decay,
    "Sigmoid": sigmoid_decay,
    "Stepped": stepped_decay
}

half_life = 10
importance = 1.0
steps_to_analyze = range(0, 21, 2)

print("Steps | Expo | Linear | Log | Sigmoid | Stepped")
print("-" * 50)

for step in steps_to_analyze:
    weights = [decay_functions[name](importance, step, half_life) for name in decay_functions.keys()]
    print(f"{step:5d} | {weights[0]:4.2f} | {weights[1]:6.2f} | {weights[2]:3.2f} | {weights[3]:7.2f} | {weights[4]:7.2f}")

# Use case scenarios for each decay function
print("\n🎯 Use Case Scenarios:")
use_cases = {
    "Exponential": "General purpose - natural forgetting pattern",
    "Linear": "Scheduled tasks - predictable timeline decay",
    "Logarithmic": "Breaking news - important initially, then stable",
    "Sigmoid": "Deadline-based - sudden relevance drop after date",
    "Stepped": "Versioned data - discrete update cycles"
}

for decay_type, use_case in use_cases.items():
    print(f"   {decay_type:12s}: {use_case}")

# Practical example: News aggregation agent
print("\n📰 Practical Example: News Aggregation Agent")
print("-" * 45)

# Different news types need different decay patterns
news_contexts = {
    "breaking_news": ContextDecay(half_life_steps=6, decay_function=logarithmic_decay),
    "market_updates": ContextDecay(half_life_steps=4, decay_function=exponential_decay),
    "feature_stories": ContextDecay(half_life_steps=12, decay_function=linear_decay),
    "scheduled_events": ContextDecay(half_life_steps=8, decay_function=sigmoid_decay)
}

# Simulate news items
news_items = [
    ("breaking_news", "major_earthquake", "7.2 earthquake hits Japan", 1.0),
    ("market_updates", "tech_stock_rise", "Tech stocks up 3% on AI news", 0.8),
    ("feature_stories", "climate_report", "Annual climate change report released", 0.7),
    ("scheduled_events", "election_results", "Election results expected tonight", 0.9),
    ("breaking_news", "celebrity_news", "Celebrity announces retirement", 0.6),
    ("market_updates", "oil_prices", "Oil prices drop 2% on supply news", 0.7)
]

print("Adding news items to specialized memory contexts:")
for category, key, description, importance in news_items:
    news_contexts[category].add(key, description, importance=importance)
    print(f"   {category:16s}: {description} (importance: {importance})")

# Simulate time passing (12 time steps)
print("\n⏰ Simulating 12 time steps...")
for step in range(12):
    for context in news_contexts.values():
        context.step()

# Analyze what's still relevant
print("\n📊 News Relevance After 12 Time Steps:")
threshold = 0.2

for category, context in news_contexts.items():
    active_news = context.get_active(threshold=threshold)
    print(f"\n{category.replace('_', ' ').title()}:")
    
    if active_news:
        for key, description, weight in active_news:
            original_importance = next(imp for cat, k, desc, imp in news_items if k == key)
            print(f"   {description[:40]:40s} (weight: {weight:.3f}, was: {original_importance})")
    else:
        print("   No items above relevance threshold")

# Insights about decay patterns
print("\n💡 Decay Pattern Insights:")
print("   📈 Logarithmic (breaking news): High initial retention, gradual decline")
print("   📉 Exponential (market updates): Steady natural decay")
print("   📊 Linear (feature stories): Predictable steady decline")
print("   📋 Sigmoid (scheduled events): Sharp drop after relevance period")

# Best practices for custom decay functions
print("\n🎯 Best Practices for Custom Decay Functions:")
best_practices = [
    "Match decay pattern to information lifecycle",
    "Consider domain-specific relevance patterns",
    "Test decay functions with realistic time scales",
    "Balance retention with memory efficiency",
    "Document decay rationale for team understanding"
]

for practice in best_practices:
    print(f"   • {practice}")

## 💰 Chapter 3: Cost-Optimized Memory Management

Learn to balance memory richness with budget constraints using Argentum's cost optimization features.

In [ ]:
print("💰 Chapter 3: Cost-Optimized Memory Management")
print("="*50)

print("🎯 Scenario: Cost-conscious customer service agent")
print("Challenge: Maintain service quality while controlling memory costs")

# Initialize cost-optimized context manager
cost_optimized_memory = ContextDecay(
    half_life_steps=10,
    cost_optimization=True,
    max_context_cost=0.50  # 50 cents maximum
)

print(f"\n💸 Initialized with ${cost_optimized_memory._max_context_cost:.2f} budget")

# Simulate customer service interactions with varying storage costs
print("\n🤝 Customer Service Interaction Simulation:")

customer_interactions = [
    # (key, value, importance, storage_cost, description)
    ("customer_name", "Jennifer Martinez", 0.9, 0.02, "Customer identification"),
    ("issue_type", "billing_discrepancy", 1.0, 0.03, "Primary issue to resolve"),
    ("account_history", "premium_member_3_years", 0.8, 0.05, "Account context"),
    ("previous_issues", ["late_payment_2023", "upgrade_request_2024"], 0.6, 0.08, "Historical context"),
    ("current_bill", {"amount": 89.99, "due_date": "2024-11-15", "disputed_charges": ["tax_error"]}, 0.95, 0.12, "Current billing details"),
    ("customer_sentiment", "frustrated_but_polite", 0.7, 0.04, "Interaction tone"),
    ("resolution_attempts", [], 0.8, 0.06, "Troubleshooting history"),
    ("agent_notes", "customer_very_reasonable_easy_to_work_with", 0.5, 0.07, "Agent observations"),
    ("escalation_path", "supervisor_if_no_resolution", 0.6, 0.05, "Next steps if needed"),
    ("resolution_status", "investigating", 0.9, 0.04, "Current status"),
    ("callback_preference", "email_preferred_phone_backup", 0.4, 0.06, "Communication preferences"),
    ("system_logs", ["login_09:30", "bill_viewed_09:31", "dispute_filed_09:32"], 0.3, 0.15, "Technical logs"),
    ("related_customers", ["same_household_account", "family_plan_member"], 0.4, 0.10, "Related accounts"),
    ("payment_history", "excellent_never_late", 0.7, 0.08, "Payment reliability"),
    ("special_offers", "retention_discount_available", 0.2, 0.09, "Available promotions")
]

print(f"Processing {len(customer_interactions)} pieces of customer context:")
total_attempted_cost = 0

for key, value, importance, storage_cost, description in customer_interactions:
    total_attempted_cost += storage_cost
    print(f"   Adding: {description} (${storage_cost:.3f}, importance: {importance})")
    
    # Add to memory with cost tracking
    cost_optimized_memory.add(key, value, importance=importance, storage_cost=storage_cost)
    
    # Check current cost status
    current_cost = cost_optimized_memory._current_cost
    if current_cost > cost_optimized_memory._max_context_cost * 0.8:
        print(f"      ⚠️  Approaching budget limit: ${current_cost:.3f}")

print(f"\n💰 Cost Management Results:")
print(f"   Attempted total cost: ${total_attempted_cost:.3f}")
print(f"   Actual cost: ${cost_optimized_memory._current_cost:.3f}")
print(f"   Budget: ${cost_optimized_memory._max_context_cost:.3f}")
print(f"   Cost savings: ${total_attempted_cost - cost_optimized_memory._current_cost:.3f}")

# Analyze what was kept vs. pruned
active_context = cost_optimized_memory.get_active(threshold=0.1)
print(f"\n🧠 Memory Analysis:")
print(f"   Items stored: {len(active_context)}/{len(customer_interactions)}")
print(f"   Budget utilization: {(cost_optimized_memory._current_cost / cost_optimized_memory._max_context_cost) * 100:.1f}%")

print("\n📋 Context Retained (sorted by cost effectiveness):")
cost_effectiveness_data = []
for key, value, weight in active_context:
    # Find original data
    original_data = next((item for item in customer_interactions if item[0] == key), None)
    if original_data:
        _, _, importance, storage_cost, description = original_data
        cost_effectiveness = importance / storage_cost if storage_cost > 0 else importance
        cost_effectiveness_data.append((description, importance, storage_cost, cost_effectiveness, weight))

# Sort by cost effectiveness
cost_effectiveness_data.sort(key=lambda x: x[3], reverse=True)

for description, importance, storage_cost, cost_eff, weight in cost_effectiveness_data:
    print(f"   {description[:35]:35s} | Imp: {importance:.1f} | Cost: ${storage_cost:.3f} | Eff: {cost_eff:.1f} | Weight: {weight:.3f}")

# Get detailed cost report
cost_report = cost_optimized_memory.get_cost_report()
print(f"\n📊 Detailed Cost Report:")
print(f"   Total items: {cost_report['total_items']}")
print(f"   Total cost: ${cost_report['total_cost']:.3f}")
print(f"   Budget utilization: {cost_report['budget_utilization']:.1%}")
print(f"   Average cost per item: ${cost_report['average_cost_per_item']:.3f}")
print(f"   Cost efficiency: {cost_report['cost_efficiency']:.1f} importance/$")
print(f"   Items pruned: {cost_report['items_pruned']}")
print(f"   Cost saved through pruning: ${cost_report['cost_saved']:.3f}")

# Demonstrate dynamic cost adjustment
print("\n⚖️ Dynamic Cost Adjustment Demo:")
print("Simulating memory pressure with high-cost items...")

# Try to add expensive items
expensive_items = [
    ("full_call_transcript", "[Very long transcript with 1000+ words...]", 0.8, 0.25),
    ("detailed_product_catalog", "[Complete product database cache]", 0.3, 0.20),
    ("ai_model_cache", "[Cached model weights and embeddings]", 0.4, 0.30)
]

for key, value, importance, storage_cost in expensive_items:
    initial_cost = cost_optimized_memory._current_cost
    cost_optimized_memory.add(key, value, importance=importance, storage_cost=storage_cost)
    final_cost = cost_optimized_memory._current_cost
    
    cost_change = final_cost - initial_cost
    print(f"   {key}: Attempted ${storage_cost:.3f}, actual cost change: ${cost_change:.3f}")

# Final memory state
final_active = cost_optimized_memory.get_active(threshold=0.1)
final_cost_report = cost_optimized_memory.get_cost_report()

print(f"\n🎯 Final Memory State:")
print(f"   Active items: {len(final_active)}")
print(f"   Total cost: ${final_cost_report['total_cost']:.3f}")
print(f"   Budget compliance: {'✅' if final_cost_report['budget_utilization'] <= 1.0 else '❌'}")
print(f"   Cost efficiency: {final_cost_report['cost_efficiency']:.1f} importance/$")

# Cost optimization strategies
print("\n💡 Cost Optimization Strategies Demonstrated:")
strategies = [
    "Automatic pruning of low cost-effectiveness items",
    "Importance-based budget allocation",
    "Dynamic cost adjustment for high-value items",
    "Real-time budget monitoring and compliance",
    "Cost-effectiveness ranking for retention decisions"
]

for strategy in strategies:
    print(f"   ✅ {strategy}")

# Production recommendations
print("\n🚀 Production Cost Management Recommendations:")
recommendations = [
    "Set budget limits based on business value of memory retention",
    "Monitor cost efficiency metrics in real-time",
    "Implement tiered storage for different importance levels",
    "Use cost forecasting to predict memory budget needs",
    "Regular cost audits to optimize storage strategies"
]

for rec in recommendations:
    print(f"   📋 {rec}")

## 🎪 Chapter 4: Multi-Agent Memory Coordination

Learn to manage shared memory contexts across multiple agents with different decay requirements.

In [ ]:
print("🎪 Chapter 4: Multi-Agent Memory Coordination")
print("="*50)

print("🎯 Scenario: Multi-agent customer support system")
print("Agents: Intake → Specialist → Follow-up")

# Define different agent types with specialized memory needs
class MultiAgentMemoryCoordinator:
    def __init__(self):
        # Different agents have different memory priorities
        self.agents = {
            "intake_agent": ContextDecay(
                half_life_steps=5,  # Fast turnover, handles many customers
                cost_optimization=True,
                max_context_cost=0.25
            ),
            "specialist_agent": ContextDecay(
                half_life_steps=15,  # Longer memory for complex problem-solving
                cost_optimization=True,
                max_context_cost=0.75
            ),
            "followup_agent": ContextDecay(
                half_life_steps=20,  # Longest memory for relationship building
                cost_optimization=True,
                max_context_cost=0.50
            )
        }
        
        # Shared context that all agents can access
        self.shared_context = ContextDecay(
            half_life_steps=25,  # Long-term shared memory
            cost_optimization=True,
            max_context_cost=1.00
        )
        
    def transfer_context(self, from_agent: str, to_agent: str, context_keys: list):
        """Transfer context between agents with importance adjustment"""
        source = self.agents[from_agent]
        target = self.agents[to_agent]
        
        transferred = []
        for key in context_keys:
            if key in source._items:
                item = source._items[key]
                current_weight = source._calculate_current_weight(item)
                
                # Adjust importance based on target agent's needs
                importance_multiplier = self._get_importance_multiplier(from_agent, to_agent, key)
                adjusted_importance = min(1.0, item['importance'] * importance_multiplier)
                
                # Transfer to target agent
                target.add(
                    key, 
                    item['value'], 
                    importance=adjusted_importance,
                    storage_cost=item.get('storage_cost', 0.02)
                )
                
                transferred.append((key, current_weight, adjusted_importance))
        
        return transferred
    
    def _get_importance_multiplier(self, from_agent: str, to_agent: str, key: str) -> float:
        """Adjust importance based on agent transition"""
        # Define what each agent type values most
        agent_priorities = {
            "intake_agent": ["customer_info", "issue_type", "urgency"],
            "specialist_agent": ["technical_details", "troubleshooting_history", "solution_attempts"],
            "followup_agent": ["resolution_status", "customer_satisfaction", "follow_up_schedule"]
        }
        
        # Boost importance if the key is high priority for target agent
        if any(priority in key for priority in agent_priorities.get(to_agent, [])):
            return 1.2  # 20% boost
        
        # Reduce importance if it was specific to source agent
        if any(priority in key for priority in agent_priorities.get(from_agent, [])):
            return 0.8  # 20% reduction
        
        return 1.0  # No change
    
    def add_to_shared(self, key: str, value, importance: float, storage_cost: float = 0.05):
        """Add important context to shared memory"""
        self.shared_context.add(key, value, importance=importance, storage_cost=storage_cost)
    
    def get_agent_stats(self):
        """Get statistics for all agents"""
        stats = {}
        for agent_name, agent in self.agents.items():
            stats[agent_name] = {
                "memory_stats": agent.get_stats(),
                "cost_report": agent.get_cost_report(),
                "active_items": len(agent.get_active(threshold=0.2))
            }
        
        stats["shared"] = {
            "memory_stats": self.shared_context.get_stats(),
            "cost_report": self.shared_context.get_cost_report(),
            "active_items": len(self.shared_context.get_active(threshold=0.2))
        }
        
        return stats

# Initialize the coordinator
coordinator = MultiAgentMemoryCoordinator()
print("✅ Multi-agent memory coordinator initialized")

# Simulate customer support workflow
print("\n🎬 Customer Support Workflow Simulation:")
print("-" * 45)

# Phase 1: Intake Agent
print("\n1️⃣ Intake Agent Phase:")
intake_data = [
    ("customer_info", "David Chen, Account #12789", 0.9, 0.03),
    ("issue_type", "internet_connectivity_problem", 1.0, 0.02),
    ("urgency", "high_business_critical", 0.95, 0.02),
    ("customer_tier", "enterprise_platinum", 0.8, 0.04),
    ("initial_symptoms", "intermittent_disconnections_since_yesterday", 0.7, 0.05),
    ("location", "downtown_office_building", 0.6, 0.03),
    ("contact_preference", "phone_immediate_response", 0.8, 0.02)
]

for key, value, importance, cost in intake_data:
    coordinator.agents["intake_agent"].add(key, value, importance=importance, storage_cost=cost)
    print(f"   Added: {key} (importance: {importance})")

# Add critical info to shared context
coordinator.add_to_shared("customer_info", "David Chen, Account #12789", 0.95, 0.04)
coordinator.add_to_shared("issue_type", "internet_connectivity_problem", 1.0, 0.03)
coordinator.add_to_shared("urgency", "high_business_critical", 0.9, 0.02)

# Advance time (intake agent processes quickly)
for _ in range(3):
    coordinator.agents["intake_agent"].step()

# Phase 2: Transfer to Specialist
print("\n2️⃣ Transfer to Specialist Agent:")
transfer_keys = ["customer_info", "issue_type", "urgency", "customer_tier", "initial_symptoms"]
transferred = coordinator.transfer_context("intake_agent", "specialist_agent", transfer_keys)

for key, old_weight, new_importance in transferred:
    print(f"   Transferred: {key} (weight: {old_weight:.3f} → importance: {new_importance:.3f})")

# Specialist adds technical context
print("\n🔧 Specialist Agent Phase:")
specialist_data = [
    ("technical_details", "DNS_resolution_failures_detected", 0.9, 0.06),
    ("troubleshooting_history", ["ping_test_failed", "traceroute_timeout", "DNS_flush_attempted"], 0.8, 0.08),
    ("solution_attempts", ["router_restart", "DNS_server_change"], 0.85, 0.07),
    ("root_cause", "ISP_DNS_server_maintenance", 0.95, 0.05),
    ("estimated_resolution", "2_hours_ISP_completion", 0.9, 0.04),
    ("workaround_provided", "temporary_public_DNS_configuration", 0.8, 0.06)
]

for key, value, importance, cost in specialist_data:
    coordinator.agents["specialist_agent"].add(key, value, importance=importance, storage_cost=cost)
    print(f"   Added: {key} (importance: {importance})")

# Update shared context with resolution info
coordinator.add_to_shared("root_cause", "ISP_DNS_server_maintenance", 0.95, 0.05)
coordinator.add_to_shared("solution_status", "workaround_provided_monitoring", 0.9, 0.04)

# Advance time (specialist works longer)
for _ in range(8):
    coordinator.agents["specialist_agent"].step()

# Phase 3: Transfer to Follow-up
print("\n3️⃣ Transfer to Follow-up Agent:")
followup_transfer_keys = ["customer_info", "root_cause", "solution_attempts", "estimated_resolution"]
transferred_followup = coordinator.transfer_context("specialist_agent", "followup_agent", followup_transfer_keys)

for key, old_weight, new_importance in transferred_followup:
    print(f"   Transferred: {key} (weight: {old_weight:.3f} → importance: {new_importance:.3f})")

# Follow-up agent adds relationship context
print("\n📞 Follow-up Agent Phase:")
followup_data = [
    ("resolution_status", "fully_resolved_customer_satisfied", 1.0, 0.03),
    ("customer_satisfaction", "9_out_of_10_rating", 0.9, 0.04),
    ("follow_up_schedule", "1_week_check_in_scheduled", 0.8, 0.03),
    ("relationship_notes", "appreciates_proactive_communication", 0.7, 0.05),
    ("future_opportunities", "interested_in_redundancy_solutions", 0.6, 0.06)
]

for key, value, importance, cost in followup_data:
    coordinator.agents["followup_agent"].add(key, value, importance=importance, storage_cost=cost)
    print(f"   Added: {key} (importance: {importance})")

# Final shared context update
coordinator.add_to_shared("case_resolution", "successful_high_satisfaction", 1.0, 0.04)
coordinator.add_to_shared("customer_satisfaction", "9_out_of_10_rating", 0.9, 0.03)

# Advance time for follow-up
for _ in range(5):
    coordinator.agents["followup_agent"].step()
    coordinator.shared_context.step()

# Analyze final memory state across all agents
print("\n📊 Final Multi-Agent Memory Analysis:")
print("-" * 40)

stats = coordinator.get_agent_stats()

for agent_name, agent_stats in stats.items():
    memory_stats = agent_stats["memory_stats"]
    cost_report = agent_stats["cost_report"]
    
    print(f"\n{agent_name.replace('_', ' ').title()}:")
    print(f"   Active items: {agent_stats['active_items']}")
    print(f"   Total items: {memory_stats['total_items']}")
    print(f"   Average decay: {memory_stats['avg_decay']:.3f}")
    
    if cost_report:
        print(f"   Total cost: ${cost_report['total_cost']:.3f}")
        print(f"   Budget utilization: {cost_report['budget_utilization']:.1%}")
        print(f"   Cost efficiency: {cost_report['cost_efficiency']:.1f}")

# Demonstrate memory coordination benefits
print("\n💡 Multi-Agent Memory Coordination Benefits:")
benefits = [
    "Context preservation across agent handoffs",
    "Specialized memory patterns for different agent types",
    "Shared knowledge base for critical information",
    "Cost optimization per agent's operational needs",
    "Importance adjustment based on agent priorities"
]

for benefit in benefits:
    print(f"   ✅ {benefit}")

# Show shared context effectiveness
shared_active = coordinator.shared_context.get_active(threshold=0.3)
print(f"\n🌐 Shared Context Effectiveness:")
print(f"   Shared memories retained: {len(shared_active)}")
print(f"   Available to all agents for future cases")
for key, value, weight in shared_active:
    print(f"   {key}: {weight:.3f} weight")

## ⚡ Chapter 5: Performance Optimization & Scaling

Learn to optimize ContextDecay for high-performance production environments.

In [ ]:
print("⚡ Chapter 5: Performance Optimization & Scaling")
print("="*55)

print("🎯 High-Performance Context Management for Production")

# Performance-optimized context manager
class HighPerformanceContextManager:
    def __init__(self, agent_id: str):
        self.agent_id = agent_id
        self.context = ContextDecay(
            half_life_steps=12,
            cost_optimization=True,
            max_context_cost=2.00
        )
        
        # Performance tracking
        self.performance_metrics = {
            "add_operations": 0,
            "get_active_operations": 0,
            "cleanup_operations": 0,
            "total_add_time": 0.0,
            "total_query_time": 0.0,
            "total_cleanup_time": 0.0,
            "memory_pressure_events": 0
        }
        
        # Configuration
        self.config = {
            "cleanup_interval": 50,      # Clean up every 50 operations
            "batch_size": 20,            # Process items in batches
            "memory_pressure_threshold": 1000,  # Max items before pressure
            "performance_warning_ms": 100      # Warn if operations take > 100ms
        }
    
    def add_context(self, key: str, value, importance: float, storage_cost: float = 0.01):
        """Add context with performance monitoring"""
        start_time = time.time()
        
        # Check memory pressure
        if len(self.context._items) > self.config["memory_pressure_threshold"]:
            self.performance_metrics["memory_pressure_events"] += 1
            self._emergency_cleanup()
        
        # Add to context
        self.context.add(key, value, importance=importance, storage_cost=storage_cost)
        
        # Track performance
        operation_time = (time.time() - start_time) * 1000  # ms
        self.performance_metrics["add_operations"] += 1
        self.performance_metrics["total_add_time"] += operation_time
        
        if operation_time > self.config["performance_warning_ms"]:
            print(f"⚠️  Slow add operation: {operation_time:.1f}ms for {key}")
        
        # Periodic cleanup
        if self.performance_metrics["add_operations"] % self.config["cleanup_interval"] == 0:
            self._scheduled_cleanup()
    
    def get_relevant_context(self, threshold: float = 0.3, max_items: int = 100):
        """Get relevant context with performance optimization"""
        start_time = time.time()
        
        # Get active context
        active_context = self.context.get_active(threshold=threshold)
        
        # Limit results for performance
        if len(active_context) > max_items:
            active_context = active_context[:max_items]
        
        # Track performance
        operation_time = (time.time() - start_time) * 1000
        self.performance_metrics["get_active_operations"] += 1
        self.performance_metrics["total_query_time"] += operation_time
        
        if operation_time > self.config["performance_warning_ms"]:
            print(f"⚠️  Slow query operation: {operation_time:.1f}ms")
        
        return active_context
    
    def _scheduled_cleanup(self):
        """Scheduled cleanup to maintain performance"""
        start_time = time.time()
        
        removed = self.context.clear_expired(threshold=0.1)
        
        operation_time = (time.time() - start_time) * 1000
        self.performance_metrics["cleanup_operations"] += 1
        self.performance_metrics["total_cleanup_time"] += operation_time
        
        if removed > 0:
            print(f"🧹 Scheduled cleanup: removed {removed} expired items ({operation_time:.1f}ms)")
    
    def _emergency_cleanup(self):
        """Emergency cleanup under memory pressure"""
        print(f"🚨 Emergency cleanup triggered: {len(self.context._items)} items")
        
        # More aggressive cleanup
        removed = self.context.clear_expired(threshold=0.2)
        print(f"   Removed {removed} items with threshold 0.2")
        
        # If still too many items, remove by cost effectiveness
        if len(self.context._items) > self.config["memory_pressure_threshold"] * 0.8:
            self.context._prune_by_cost_effectiveness()
            print(f"   Applied cost-based pruning")
    
    def get_performance_report(self):
        """Get detailed performance metrics"""
        metrics = self.performance_metrics.copy()
        
        # Calculate averages
        if metrics["add_operations"] > 0:
            metrics["avg_add_time_ms"] = metrics["total_add_time"] / metrics["add_operations"]
        
        if metrics["get_active_operations"] > 0:
            metrics["avg_query_time_ms"] = metrics["total_query_time"] / metrics["get_active_operations"]
        
        if metrics["cleanup_operations"] > 0:
            metrics["avg_cleanup_time_ms"] = metrics["total_cleanup_time"] / metrics["cleanup_operations"]
        
        # Add context stats
        metrics["current_items"] = len(self.context._items)
        metrics["current_cost"] = self.context._current_cost
        
        return metrics

# Initialize high-performance manager
perf_manager = HighPerformanceContextManager("high_freq_trading_agent")
print("✅ High-performance context manager initialized")

# Simulate high-frequency operations
print("\n🚀 High-Frequency Operation Simulation:")
print("Simulating 200 rapid context additions...")

# Generate realistic high-frequency data
operation_types = [
    ("market_tick", 0.6, 0.001),      # High volume, low importance, low cost
    ("trade_signal", 0.9, 0.005),     # Medium volume, high importance, low cost
    ("risk_update", 0.8, 0.003),      # Medium volume, high importance, low cost
    ("news_event", 0.7, 0.010),       # Low volume, medium importance, medium cost
    ("model_update", 0.95, 0.020),    # Very low volume, very high importance, high cost
]

# Simulate operations
start_simulation = time.time()

for i in range(200):
    # Choose operation type with realistic distribution
    if i % 20 == 0:  # Model update every 20 operations
        op_type, importance, cost = operation_types[4]
        key = f"{op_type}_{i}"
        value = f"Model weights updated at step {i}"
    elif i % 10 == 0:  # News event every 10 operations
        op_type, importance, cost = operation_types[3]
        key = f"{op_type}_{i}"
        value = f"Market news: {random.choice(['Fed announcement', 'Earnings report', 'Economic data'])}"
    elif i % 3 == 0:  # Risk update every 3 operations
        op_type, importance, cost = operation_types[2]
        key = f"{op_type}_{i}"
        value = f"Portfolio risk: {random.uniform(0.1, 0.8):.3f}"
    elif i % 2 == 0:  # Trade signal every 2 operations
        op_type, importance, cost = operation_types[1]
        key = f"{op_type}_{i}"
        value = f"Signal: {random.choice(['BUY', 'SELL', 'HOLD'])} confidence {random.uniform(0.6, 0.95):.2f}"
    else:  # Market tick for the rest
        op_type, importance, cost = operation_types[0]
        key = f"{op_type}_{i}"
        value = f"Price: ${random.uniform(100, 200):.2f}"
    
    # Add slight randomness to importance and cost
    actual_importance = importance + random.uniform(-0.1, 0.1)
    actual_importance = max(0.1, min(1.0, actual_importance))
    
    actual_cost = cost * random.uniform(0.8, 1.2)
    
    # Add to context
    perf_manager.add_context(key, value, actual_importance, actual_cost)
    
    # Advance time occasionally
    if i % 10 == 0:
        perf_manager.context.step()
    
    # Occasional queries
    if i % 25 == 0:
        relevant = perf_manager.get_relevant_context(threshold=0.4, max_items=50)
        if i % 100 == 0:  # Less frequent logging
            print(f"   Step {i}: {len(relevant)} relevant items retrieved")

total_simulation_time = time.time() - start_simulation
print(f"\n⏱️  Simulation completed in {total_simulation_time:.3f} seconds")

# Performance analysis
print("\n📊 Performance Analysis:")
perf_report = perf_manager.get_performance_report()

print(f"   Operations completed:")
print(f"     Add operations: {perf_report['add_operations']}")
print(f"     Query operations: {perf_report['get_active_operations']}")
print(f"     Cleanup operations: {perf_report['cleanup_operations']}")

print(f"\n   Average operation times:")
print(f"     Add: {perf_report.get('avg_add_time_ms', 0):.2f}ms")
print(f"     Query: {perf_report.get('avg_query_time_ms', 0):.2f}ms")
print(f"     Cleanup: {perf_report.get('avg_cleanup_time_ms', 0):.2f}ms")

print(f"\n   Memory management:")
print(f"     Current items: {perf_report['current_items']}")
print(f"     Current cost: ${perf_report['current_cost']:.3f}")
print(f"     Memory pressure events: {perf_report['memory_pressure_events']}")

print(f"\n   Throughput:")
operations_per_second = perf_report['add_operations'] / total_simulation_time
print(f"     Add operations/sec: {operations_per_second:.1f}")

# Stress test - burst operations
print("\n🔥 Stress Test: Burst Operations")
print("Simulating 100 rapid-fire additions...")

burst_start = time.time()
for i in range(100):
    key = f"burst_op_{i}"
    value = f"Burst data {i}"
    importance = random.uniform(0.3, 0.9)
    cost = random.uniform(0.001, 0.005)
    
    perf_manager.add_context(key, value, importance, cost)

burst_time = time.time() - burst_start
burst_ops_per_sec = 100 / burst_time

print(f"   Burst completed in {burst_time:.3f} seconds")
print(f"   Burst throughput: {burst_ops_per_sec:.1f} ops/sec")

# Final performance assessment
final_report = perf_manager.get_performance_report()
final_relevant = perf_manager.get_relevant_context(threshold=0.3)

print(f"\n🎯 Final Performance Assessment:")
print(f"   Total operations: {final_report['add_operations']}")
print(f"   Items retained: {len(final_relevant)}")
print(f"   Retention rate: {(len(final_relevant) / final_report['add_operations']) * 100:.1f}%")
print(f"   Memory efficiency: {final_report['current_cost'] / len(final_relevant):.4f} $/item")

# Production optimization recommendations
print("\n🚀 Production Optimization Recommendations:")
recommendations = [
    f"✅ Achieved {operations_per_second:.0f} ops/sec sustained throughput",
    f"✅ Memory pressure handled with {final_report['memory_pressure_events']} emergency cleanups",
    f"✅ Average add latency: {final_report.get('avg_add_time_ms', 0):.1f}ms (target: <10ms)",
    f"✅ Query performance: {final_report.get('avg_query_time_ms', 0):.1f}ms (target: <50ms)",
    f"✅ Cost efficiency: ${final_report['current_cost']:.3f} for {len(final_relevant)} items"
]

for rec in recommendations:
    print(f"   {rec}")

print("\n⚡ Performance optimization strategies validated:")
strategies = [
    "Periodic automated cleanup prevents memory bloat",
    "Emergency cleanup handles unexpected memory pressure",
    "Cost-based pruning maintains budget compliance",
    "Performance monitoring identifies bottlenecks",
    "Batch processing optimizes query operations"
]

for strategy in strategies:
    print(f"   📈 {strategy}")

print(f"\n🏆 Ready for production deployment at scale!")

## 🎯 Chapter 6: Real-World Applications & Best Practices

Explore practical applications and proven patterns for deploying ContextDecay in production systems.

In [ ]:
print("🎯 Chapter 6: Real-World Applications & Best Practices")
print("="*60)

print("📚 Comprehensive Application Examples:")

# Application 1: E-commerce Recommendation Engine
print("\n🛒 Application 1: E-commerce Recommendation Engine")
print("-" * 50)

ecommerce_memory = ContextDecay(
    half_life_steps=15,  # 15 user actions
    cost_optimization=True,
    max_context_cost=1.50
)

# Simulate user shopping behavior
shopping_actions = [
    ("viewed_product", "laptop_dell_xps13", 0.6, 0.02, "Product view"),
    ("added_to_cart", "laptop_dell_xps13", 0.9, 0.05, "Strong purchase intent"),
    ("viewed_category", "electronics_laptops", 0.7, 0.03, "Category interest"),
    ("search_query", "lightweight laptop programming", 0.8, 0.04, "User intent"),
    ("price_filter", "1000_2000_range", 0.8, 0.02, "Budget constraint"),
    ("viewed_reviews", "laptop_dell_xps13_reviews", 0.7, 0.06, "Research behavior"),
    ("compared_products", ["laptop_dell_xps13", "macbook_air_m2"], 0.85, 0.08, "Active comparison"),
    ("abandoned_cart", "session_timeout", 0.3, 0.01, "Session ended"),
    ("returned_later", "same_day_return", 0.9, 0.03, "High intent"),
    ("purchased", "laptop_dell_xps13", 1.0, 0.10, "Conversion!"),
    ("browsed_accessories", "laptop_cases_mice", 0.6, 0.04, "Cross-sell opportunity")
]

print("User shopping journey:")
for i, (key, value, importance, cost, description) in enumerate(shopping_actions):
    ecommerce_memory.add(key, value, importance=importance, storage_cost=cost)
    ecommerce_memory.step()  # Each action advances time
    print(f"   {i+1:2d}. {description}: {key}")

# Analyze shopping behavior retention
shopping_context = ecommerce_memory.get_active(threshold=0.3)
print(f"\nRecommendation context (threshold 0.3): {len(shopping_context)} items")
for key, value, weight in shopping_context:
    print(f"   {key}: {weight:.3f} weight")

# Application 2: Medical Diagnosis Assistant
print("\n🏥 Application 2: Medical Diagnosis Assistant")
print("-" * 45)

# Custom decay for medical context - longer retention for critical info
def medical_decay(importance, steps, half_life):
    """Medical context decay - critical symptoms persist longer"""
    if importance > 0.9:  # Critical symptoms
        return importance * (0.5 ** (steps / (half_life * 2)))
    elif importance > 0.7:  # Important symptoms
        return importance * (0.5 ** (steps / (half_life * 1.5)))
    else:  # General information
        return importance * (0.5 ** (steps / half_life))

medical_memory = ContextDecay(
    half_life_steps=20,  # Longer retention for medical context
    decay_function=medical_decay,
    cost_optimization=True,
    max_context_cost=3.00  # Higher budget for critical application
)

# Simulate patient consultation
consultation_data = [
    ("chief_complaint", "chest_pain_shortness_of_breath", 1.0, 0.15, "Primary concern"),
    ("symptom_duration", "3_days_worsening", 0.95, 0.08, "Timeline critical"),
    ("pain_severity", "7_out_of_10", 0.9, 0.05, "Pain assessment"),
    ("medical_history", "hypertension_diabetes", 0.9, 0.12, "Risk factors"),
    ("current_medications", ["metformin", "lisinopril"], 0.85, 0.10, "Drug interactions"),
    ("vital_signs", "BP_150_95_HR_105_irregular", 0.95, 0.08, "Objective data"),
    ("family_history", "father_heart_attack_age_55", 0.8, 0.07, "Genetic risk"),
    ("lifestyle_factors", "smoker_sedentary", 0.7, 0.06, "Risk modifiers"),
    ("previous_tests", "EKG_6_months_ago_normal", 0.6, 0.09, "Historical data"),
    ("physical_exam", "heart_murmur_detected", 0.9, 0.11, "Clinical findings"),
    ("differential_diagnosis", ["MI", "angina", "arrhythmia"], 0.95, 0.15, "Working diagnosis"),
    ("recommended_tests", ["EKG", "troponin", "echo"], 0.9, 0.12, "Next steps")
]

print("Patient consultation progression:")
for key, value, importance, cost, description in consultation_data:
    medical_memory.add(key, value, importance=importance, storage_cost=cost)
    print(f"   {description}: importance {importance}")

# Simulate time progression (several follow-up visits)
for _ in range(10):
    medical_memory.step()

# Analyze medical context retention
medical_context = medical_memory.get_active(threshold=0.4)
print(f"\nCritical medical context retained: {len(medical_context)} items")
critical_items = [item for item in medical_context if item[2] > 0.7]
print(f"High-priority items (weight > 0.7): {len(critical_items)}")

# Application 3: Financial Trading Bot
print("\n💰 Application 3: Financial Trading Bot")
print("-" * 40)

# Multi-timeframe memory for different types of financial data
trading_memory = {
    "tick_data": ContextDecay(half_life_steps=3),      # Very short-term
    "technical_signals": ContextDecay(half_life_steps=10),  # Short-term
    "fundamental_data": ContextDecay(half_life_steps=50),   # Long-term
    "risk_metrics": ContextDecay(half_life_steps=20)       # Medium-term
}

# Simulate trading day
trading_events = [
    ("tick_data", "price_spike", "AAPL_sudden_jump_2%", 0.8),
    ("technical_signals", "rsi_overbought", "RSI_above_70", 0.9),
    ("fundamental_data", "earnings_beat", "Q3_earnings_15%_above_estimate", 0.95),
    ("risk_metrics", "portfolio_var", "VaR_increased_to_2.5%", 0.85),
    ("tick_data", "volume_surge", "trading_volume_3x_average", 0.7),
    ("technical_signals", "breakout_confirmed", "resistance_level_broken", 0.9),
    ("fundamental_data", "analyst_upgrade", "Goldman_raises_target_to_180", 0.8),
    ("risk_metrics", "correlation_change", "correlation_with_SPY_decreased", 0.6)
]

print("Trading day events:")
for category, key, value, importance in trading_events:
    trading_memory[category].add(key, value, importance=importance)
    print(f"   {category}: {key} (importance: {importance})")

# Advance time for different categories
for _ in range(15):
    for memory in trading_memory.values():
        memory.step()

# Analyze what persists in each timeframe
print("\nMemory persistence by timeframe:")
for category, memory in trading_memory.items():
    active = memory.get_active(threshold=0.3)
    print(f"   {category}: {len(active)} items still relevant")

# Best Practices Summary
print("\n🎯 Best Practices Summary:")
print("="*30)

best_practices = {
    "🔧 Configuration": [
        "Match half-life to domain-specific information lifecycle",
        "Set importance scores based on business impact",
        "Configure cost budgets aligned with operational constraints",
        "Choose decay functions that match information aging patterns"
    ],
    
    "⚡ Performance": [
        "Monitor memory usage and implement automatic cleanup",
        "Use cost optimization to balance quality and efficiency",
        "Batch operations when possible for better throughput",
        "Set up alerts for memory pressure and performance degradation"
    ],
    
    "🏭 Production": [
        "Implement gradual rollout with A/B testing",
        "Monitor business metrics impact, not just technical metrics",
        "Have fallback strategies for memory system failures",
        "Regular audits of memory patterns and cost efficiency"
    ],
    
    "🔒 Security & Compliance": [
        "Implement data retention policies for sensitive information",
        "Ensure memory cleanup meets regulatory requirements",
        "Audit trails for memory management decisions",
        "Privacy-compliant handling of personal data in memory"
    ]
}

for category, practices in best_practices.items():
    print(f"\n{category}:")
    for practice in practices:
        print(f"   • {practice}")

# Common Pitfalls and Solutions
print("\n❌ Common Pitfalls and Solutions:")
pitfalls = {
    "Over-retention": "Setting half-life too long → Monitor memory growth and cost impact",
    "Under-retention": "Losing critical context → Use importance weighting and custom decay",
    "Cost explosion": "No budget limits → Implement cost optimization and monitoring",
    "Performance degradation": "Large memory footprint → Regular cleanup and memory pressure handling",
    "Poor decay matching": "Wrong decay pattern → Study domain patterns and test different functions"
}

for pitfall, solution in pitfalls.items():
    print(f"   ⚠️  {pitfall}: {solution}")

# Integration patterns
print("\n🔗 Integration Patterns:")
patterns = {
    "Microservices": "Deploy memory management as separate service with API",
    "Event-driven": "Trigger memory updates based on system events",
    "Batch processing": "Periodic memory analysis and optimization jobs",
    "Real-time": "Inline memory management with primary application logic",
    "Hybrid": "Combine real-time updates with batch optimization"
}

for pattern, description in patterns.items():
    print(f"   🏗️  {pattern}: {description}")

print("\n🏆 ContextDecay Mastery Achieved!")
print("\nYou're now equipped to:")
mastery_skills = [
    "Design memory systems that balance retention and efficiency",
    "Implement cost-optimized context management",
    "Deploy custom decay functions for domain-specific needs",
    "Scale memory management for high-performance applications",
    "Coordinate memory across multi-agent systems",
    "Monitor and optimize memory systems in production"
]

for skill in mastery_skills:
    print(f"   🎯 {skill}")

print("\n🚀 Ready to build intelligent memory systems that scale!")

## 🎓 Summary & Next Steps

Congratulations! You've mastered the art of agent memory management with ContextDecay. Here's what you've accomplished:

### 🧠 Core Concepts Mastered
- **Temporal Decay Models**: Understanding how memories naturally fade and implementing appropriate decay functions
- **Importance-Based Retention**: Keeping critical information longer while efficiently managing less important data
- **Cost Optimization**: Balancing memory richness with budget constraints through intelligent pruning
- **Performance Scaling**: Optimizing memory systems for high-throughput production environments
- **Multi-Agent Coordination**: Managing shared and specialized memory contexts across agent systems

### 🛠️ Practical Skills Developed
- Custom decay function implementation for domain-specific needs
- Cost-conscious memory management with automatic budget compliance
- High-performance memory systems with automatic cleanup and monitoring
- Multi-timeframe memory coordination for complex applications
- Production deployment patterns with monitoring and optimization

### 🌟 Real-World Applications
- **Conversational AI**: Natural memory fade for chat interactions
- **E-commerce**: User behavior tracking with relevance decay
- **Medical Systems**: Critical symptom persistence with safety requirements
- **Financial Trading**: Multi-timeframe data management for decision making
- **Customer Service**: Context transfer between specialized agents

### 📚 Continue Your Journey
1. **06_cost_alerts.ipynb** - Learn advanced cost monitoring and alerting systems
2. **01_argentum_overview.ipynb** - Review the complete platform integration
3. **Production Implementation** - Apply these concepts to your agent systems

### 🤝 Community & Resources
- **GitHub**: [https://github.com/MarsZDF/argentum](https://github.com/MarsZDF/argentum)
- **Documentation**: [https://argentum-agent.readthedocs.io](https://argentum-agent.readthedocs.io)
- **Community**: Join discussions and share your implementations

---

*You now possess the knowledge to build memory systems that are not just intelligent, but also efficient, cost-effective, and production-ready! 🧠✨*